In [3]:
from landmark_detection.utils import load_image_database
import os
import onnxruntime as ort
import cv2
import torch
import numpy as np
import time
from tqdm import tqdm

In [6]:
versions = [(0,'roxford5k'), (198,'rparis6k'), (208,'roxford5k')]
avg_times_per_version = {}  # Guardamos los tiempos promedio aquí

for (version, dataset) in versions:
    print(f"\n🔄 Ejecutando versión {version}...")

    # Paths
    PIPELINE_ONNX = os.path.join("landmark_detection", "models", f"pipeline-yolo-cvnet-sg-v{version}.onnx")
    RESULTS_PATH = os.path.join("datasets", dataset, "results", f"df_{dataset}-v{version}.pkl")
    DESCRIPTORS_PATH = os.path.join("datasets", dataset, "results", f"desc_{dataset}-v{version}.pkl")
    IMAGE_PATH = os.path.join("datasets", dataset, "jpg")
    
    # Cargar modelo y datos
    pipeline = ort.InferenceSession(PIPELINE_ONNX, providers=["CPUExecutionProvider"])
    df_result, descriptors_final, places_db = load_image_database(RESULTS_PATH, DESCRIPTORS_PATH)
    image_names = df_result[df_result['class_id'] == -1]['image_name']
    
    # Convertir base de datos a numpy
    if isinstance(places_db, torch.Tensor):
        db_np = places_db.detach().cpu().numpy().astype(np.float32)
    else:
        db_np = np.asarray(places_db, dtype=np.float32)

    # Procesar imágenes con barra de progreso
    total_time = 0
    for image_name in tqdm(image_names, desc=f"Procesando v{version}", unit="img"):
        image = os.path.join(IMAGE_PATH, image_name)
        img_bgr = cv2.imread(image)
        img_tensor = torch.as_tensor(img_bgr)

        pipeline_inputs = {
            pipeline.get_inputs()[0].name: img_tensor.numpy(),
            pipeline.get_inputs()[1].name: db_np,
        }

        start = time.time()
        pipeline.run(None, pipeline_inputs)
        end = time.time()

        total_time += end - start

    avg_time = total_time / len(image_names)
    avg_times_per_version[version] = avg_time
    print(f"⏱️ Tiempo promedio por imagen (v{version}): {avg_time:.4f} segundos")

# Mostrar todos los resultados
print("\n📊 Tiempos promedio por versión:")
for version, avg_time in avg_times_per_version.items():
    print(f"  - Versión {version}: {avg_time:.4f} s/img")

# Tiempo promedio por imagen (v223): 0.9822 segundos (inicial paris)
# Tiempo promedio por imagen (v0): 0.7163 segundos (inicial oxford)
# Tiempo promedio por imagen (v198): 0.8229 segundos (final paris)
# Tiempo promedio por imagen (v208): 0.7320 segundos (final oxford)


🔄 Ejecutando versión 0...


Procesando v0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [1:04:35<00:00,  1.31img/s]


⏱️ Tiempo promedio por imagen (v0): 0.7163 segundos

🔄 Ejecutando versión 198...


Procesando v198: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6392/6392 [1:33:05<00:00,  1.14img/s]


⏱️ Tiempo promedio por imagen (v198): 0.8229 segundos

🔄 Ejecutando versión 208...


Procesando v208: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5063/5063 [1:06:07<00:00,  1.28img/s]

⏱️ Tiempo promedio por imagen (v208): 0.7320 segundos

📊 Tiempos promedio por versión:
  - Versión 0: 0.7163 s/img
  - Versión 198: 0.8229 s/img
  - Versión 208: 0.7320 s/img
